In [2]:
import pandas as pd

# Load the dataset
file_path = 'Weather Training Data.csv'
weather_data = pd.read_csv(file_path)

# # Display the first few rows of the dataset and its summary
# weather_data.head(), weather_data.describe(), weather_data.info()


In [3]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np

# Handling missing values
# For numeric columns, we'll impute missing values with the mean of the column
numeric_columns = weather_data.select_dtypes(include=[np.number]).columns
imputer_numeric = SimpleImputer(strategy='mean')
weather_data[numeric_columns] = imputer_numeric.fit_transform(weather_data[numeric_columns])

# For categorical columns, we'll impute missing values with the mode (most frequent value) of the column
categorical_columns = weather_data.select_dtypes(include=['object']).columns
imputer_categorical = SimpleImputer(strategy='most_frequent')
weather_data[categorical_columns] = imputer_categorical.fit_transform(weather_data[categorical_columns])

# Encoding categorical variables
label_encoders = {}
for column in categorical_columns:
    label_encoders[column] = LabelEncoder()
    weather_data[column] = label_encoders[column].fit_transform(weather_data[column])

# Normalizing the data
scaler = StandardScaler()
weather_data[numeric_columns] = scaler.fit_transform(weather_data[numeric_columns])

# # Check the preprocessed data
# weather_data.head(), weather_data.describe()


In [6]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into features (X) and target variable (y)
X = weather_data.drop(columns=['RainTomorrow'])
y = weather_data['RainTomorrow']

# Splitting the data into training, validation, and test sets
# Using 70% of data for training, 15% for validation, and 15% for testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

y_train = y_train.astype('int')
y_val = y_val.astype('int')
y_test = y_test.astype('int')

# Sizes of each dataset
sizes = {
    "Training set size": X_train.shape[0],
    "Validation set size": X_val.shape[0],
    "Test set size": X_test.shape[0]
}

sizes


{'Training set size': 69661,
 'Validation set size': 14927,
 'Test set size': 14928}

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# Initialize Logistic Regression model
logreg = LogisticRegression(solver='liblinear', random_state=42)

# Parameters for tuning
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100], # Regularization parameter
    'penalty': ['l1', 'l2'] # Norm used in the penalization
}

# Grid search with cross-validation for parameter tuning
grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_val, y_val)

# Best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Train the model with the best parameters on the full training set
logreg_optimized = LogisticRegression(**best_params, solver='liblinear', random_state=42)
logreg_optimized.fit(X_train, y_train)

# Predict on the test set
y_pred = logreg_optimized.predict(X_test)

# Evaluate the optimized model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

best_params, best_score, accuracy, classification_rep


({'C': 1, 'penalty': 'l1'},
 0.842030559136383,
 0.8413049303322615,
 '              precision    recall  f1-score   support\n\n           0       0.86      0.95      0.90     11493\n           1       0.73      0.49      0.59      3435\n\n    accuracy                           0.84     14928\n   macro avg       0.80      0.72      0.74     14928\nweighted avg       0.83      0.84      0.83     14928\n')